# Performance by product --> SoM Dashboard

In [1]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Corner/Brands/SoM/data/'
CPG = 'danone' 
nielsen_cat = True #True JUST FOR RB_HYHO, DANONE, UNILEVER SO FAR!!!
past = False
SOM_CPG = CPG.title()

## Import

In [2]:
import os
import numpy as np
import pandas as pd
from IPython.display import display

pd.options.display.float_format = '{:.2f}'.format
print([x for x in os.listdir(BASE_DIR) if x.endswith(f'{CPG}.csv')])

['som_danone.csv', 'nielsen_danone.csv', 'products_danone.csv', 'nielsen_pred_danone.csv']


In [3]:
df = pd.read_csv(os.path.join(BASE_DIR,f'products_{CPG}.csv'),
                 sep='\t', encoding='UTF-16', low_memory=False)
df.sample(2)

,month,country,city name,store_name,category en,category_id,product id,product name,barcodes,package,...,sales,orders,avg ticket,users,units found,units replaced,units requested,found rate,fulfillment,frequency
8641,diciembre 2021,MX,León,HEB,Water,35,132688,Agua alcalina,{7500326104794},Botella 900 ml,...,850,7,121,6,34,0,34,100.0%,100.0%,1.17
14964,diciembre 2021,MX,San Luis Potosí,HEB,Yogurt,24,335758,Yoghurt natural,{7501040090707},125 g,...,68,3,23,3,13,0,13,100.0%,100.0%,1.00


In [4]:
# df.rename(columns={'month  / year':'month  ','sales [currency]':'sales'}, inplace=True)

In [5]:
aux_ini = df.copy()
aux_ini['sales'] = aux_ini['sales'].astype(str).str.replace(',','').astype(float)
aux_ini = aux_ini.groupby('month  ')['sales'].sum().reset_index()
aux_ini

,month,sales
0,diciembre 2021,13731504.00


In [6]:
import re
nielsen = pd.read_csv(BASE_DIR + f'nielsen_{CPG}.csv',
                      sep='\t',encoding='utf-16'
                     ).drop_duplicates(subset=['UPC','PRODUCT ID'])
display(nielsen['UPC'].head())
nielsen['UPC'] = nielsen['UPC'].apply(lambda x: re.sub('[^\d]','',str(x)))
nielsen['UPC'] = nielsen['UPC'].astype(str).str.strip().str.split().str[0].fillna(0).astype(int)
display(nielsen['UPC'].head())
display(nielsen['UPC'].head())
nielsen.sample(2)

0    75009076
1    75009083
2    75009090
3    75010065
4    75010751
Name: UPC, dtype: object

0    75009076
1    75009083
2    75009090
3    75010065
4    75010751
Name: UPC, dtype: int64

0    75009076
1    75009083
2    75009090
3    75010065
4    75010751
Name: UPC, dtype: int64

,UPC,FORMATO,PROVEEDOR GENERAL,PROVEEDOR ABIERTO,MARCA DETALLE,MARCA GENERICA,GRAMAJE,DESCRIPCION NIELSEN,PRODUCT ID,SEGMENTO,SUBSEGMENTO 1,SUBSEGMENTO 2,SUBSEGMENTO 3
245,7501020533279,INDIVIDUAL,LALA,LALA,LALA,LALA,0.25,LALA YOGHURT P/BEBER FRESA MORA 250ML NAL.,NaN,BB LIQUIDO,BASE BUSINESS,NaN,NaN
6383,7506161207294,IND,DEMAS FABRICANTES,DEMAS FABRICANTES,BEBIDIT-AZ,BEBIDIT-AZ,0.25,BEBIDIT-AZ DISNEY BEB REF MANZANA BOT PLAST 25...,NaN,BEBIDAS SABORIZADAS,BEBIDAS REFRESCANTES,NaN,NaN


## Functions

In [7]:
import time
import numpy as np
from IPython.lib.display import Audio

start = time.time()
def time_exp(x):
    minutes, seconds = np.floor(x/60), 60*(x/60-np.floor(x/60))
    print(f"{'{:.0f}'.format(minutes)} minutos con {'{:.2f}'.format(seconds)} segundos")
    
def tono(a = 1000, b = 700, play_time_seconds = 1, framerate = 4410):
    t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)*np.pi
    return Audio(np.sin(a*t) + np.sin(b*t), rate = framerate, autoplay=True)

In [8]:
def limpio(data,month_col,to_drop):
    df = data.copy()
    df[['mes','anio']] = df[month_col].str.split(expand=True)
    meses = dict(zip(['enero','febrero','marzo','abril','mayo','junio','julio',
                      'agosto','septiembre','octubre','noviembre','diciembre'],range(1,13)))
    df['mes'] = df['mes'].replace(meses)
    df.drop(columns=to_drop,inplace=True)
    df.columns = [str(x).replace('.','').replace(' ','_').lower() for x in df.columns]
    for col in ['brand_filter','cpg_filter']:
        df[f'{col[:-7]}_id'] = df[col].apply(lambda x: ','.join(re.findall('\s\((\d+)\)',str(x))))
        df[f'{col[:-7]}_name'] = df[col].apply(lambda x: re.sub('\s\(\d+\)','',str(x)))
    return df

## Transform

### Barcodes

In [9]:
df['barcodes'] = df['barcodes'].str.replace('{','').str.replace('}','')
df['barcodes'] = df['barcodes'].str.split(',',n=1).fillna('.').apply(lambda x:x[0])
df['barcodes'].sample(4)

/var/folders/j4/1p33_bh96yn8pdz3b_5t80hm0000gn/T/ipykernel_1690/2845833164.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['barcodes'] = df['barcodes'].str.replace('{','').str.replace('}','')


15928    7506443105713
7070     7501059295551
1256     7501022013809
2589     7501040090295
Name: barcodes, dtype: object

### Merge

In [10]:
df = df.astype(str).merge(nielsen.astype(str).rename(columns={'UPC':'barcodes',
                                                              'PRODUCT ID':'product id'}),
                          on=['barcodes'] if nielsen_cat else ['barcodes','product id'],
                          how='left' if nielsen_cat else 'left')
if nielsen_cat:
    df = df.rename(columns={'product id_x':'product_id'}).drop(columns='product id_y')

df.sample(2)

,month,country,city name,store_name,category en,category_id,product_id,product name,barcodes,package,...,PROVEEDOR GENERAL,PROVEEDOR ABIERTO,MARCA DETALLE,MARCA GENERICA,GRAMAJE,DESCRIPCION NIELSEN,SEGMENTO,SUBSEGMENTO 1,SUBSEGMENTO 2,SUBSEGMENTO 3
12054,diciembre 2021,MX,Puebla,Chedraui,Yogurt,24,3302411,Queso petit suisse fresa,7506443105034,Bote 440 g,...,DANONE,DANONE,DANONINO,DANONINO,0.44,DANONINO FRESA QUESO PETIT SUISSE SOLIDO BOTE ...,INFANTIL SOLIDO,INFANTIL,nan,nan
11304,diciembre 2021,MX,Monterrey,Soriana,Yogurt,24,1902077,Yoghurt natural con cereal Froot Loops,7501020563368,180 g,...,LALA,LALA,LALA GO!,LALA GO!,0.17,LALA GO! FROOT LOOPS NATURAL VASO TAPA C/CEREA...,BB SOLIDO,BASE BUSINESS,nan,nan


### CPG

In [11]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('month  ')['sales'].sum().reset_index()
aux = aux_ini.copy()
aux = aux.set_index('month  ').join(aux_fin.set_index('month  '),rsuffix='_fin')
aux['dif'] = aux['sales_fin'] - aux['sales']
aux['dif_p'] = aux['sales_fin'] / (aux['sales']+1e-10) - 1
aux

,sales,sales_fin,dif,dif_p
month,,,,
diciembre 2021,13731504.00,13731504.00,0.00,0.00


In [12]:
prod_cpg = pd.read_csv(os.path.join(BASE_DIR,'pbbc.csv'),
                       #sep='\t',encoding='utf-16'
                      )
prod_cpg.sample(4)

,n,product_id,barcodes,category_id,category_name,brand_id,brand_name,cpg_id,cpg_name
65405,1,590636,5401030602533,277,Escritura,3004,Sharpie,975,Newell Brands
4215,1,102007,7501018319021,67,Pastas secas,5069,La Moderna,1146,La Moderna MX
52101,1,344281,7501008485057,363,Respiratorios,29036,Tabcin Active,211,Bayer
108355,1,3503490,630509944330,450,Juegos de exterior y piscina,2918,Nerf,717,No usar


In [13]:
prod_cpg.drop_duplicates(('barcodes' if nielsen_cat else 'product_id'),inplace=True)
prod_cpg = prod_cpg.astype(str).rename(columns={'product_id':'product id'})
prod_cpg['cpg filter'] = prod_cpg['cpg_name']+" ("+prod_cpg['cpg_id'].astype(str)+")"
prod_cpg.sample(4)

,n,product id,barcodes,category_id,category_name,brand_id,brand_name,cpg_id,cpg_name,cpg filter
41652,1,318654,7501003309891,931,Salsas y aderezos,189,McCormick,25,Grupo Herdez,Grupo Herdez (25)
41248,1,316794,7702111801608,1076,Cuadernos,5017,Kiut,145,Norma -Carvajal,Norma -Carvajal (145)
85120,1,1290824,7501573907022,361,Analgésicos y desinflamatorios,9992,HEB,1074,HEB,HEB (1074)
77916,1,1155682,38000138720,996,Papas fritas y snacks,688,Pringles,81,Kellogg's,Kellogg's (81)


In [14]:
df = df.merge(prod_cpg[[('barcodes' if nielsen_cat else 'product id'),'cpg filter']],
              on=('barcodes' if nielsen_cat else 'product id'),how='left')

In [15]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('month  ')['sales'].sum().reset_index()
aux = aux_ini.copy()
aux = aux.set_index('month  ').join(aux_fin.set_index('month  '),rsuffix='_fin')
aux['dif'] = aux['sales_fin'] - aux['sales']
aux['dif_p'] = aux['sales_fin'] / (aux['sales']+1e-10) - 1
aux

,sales,sales_fin,dif,dif_p
month,,,,
diciembre 2021,13731504.00,13731504.00,0.00,0.00


### Limpio

In [16]:
df = limpio(df,'month  ',
            ['month  ','country ','avg ticket','found rate','fulfillment','frequency'])
df.sample(2)

,city_name,store_name,category_en,category_id,product_id,product_name,barcodes,package,product_sku,brand_filter,...,subsegmento_1,subsegmento_2,subsegmento_3,cpg_filter,mes,anio,brand_id,brand_name,cpg_id,cpg_name
7871,Guadalajara,Soriana,Yogurt,24,1397297,Yoghurt bebible surtido,7501020561258,6 x 220 g,nan,Lala (5096),...,BASE BUSINESS,nan,nan,Grupo Lala MX (7),12,2021,5096,Lala,7,Grupo Lala MX
8371,León,Chedraui,Yogurt,24,2669344,Yoghurt Batido Sin Azucar,7501040097980,950 g,443776,Yoplait (6413),...,BASE BUSINESS,nan,nan,Sigma Total (72),12,2021,6413,Yoplait,72,Sigma Total


In [17]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('mes')['sales'].sum().reset_index()
aux_fin['mes'] = aux_fin['mes'].astype(int).map(dict(zip(range(1,13),
                                               ['ene','feb','mar','abr','may','jun',
                                                'jul','ago','sep','oct','nov','dic'])))
aux = aux_ini.copy()
aux['month  '] = aux['month  '].str.split().str[0].str[:3]
aux = aux.set_index('month  ').join(aux_fin.set_index('mes'),rsuffix='_fin')
aux['dif'] = aux['sales_fin'] - aux['sales']
aux['dif_p'] = aux['sales_fin'] / (aux['sales']+1e-10) - 1
aux

,sales,sales_fin,dif,dif_p
month,,,,
dic,13731504.00,13731504.00,0.00,0.00


In [18]:
aux = df[df['formato'].astype(str)=='nan'].copy()
df = df[~(df['formato'].astype(str)=='nan')].copy()
null_fill = {'proveedor_general':'cpg_name',
             'proveedor_abierto':'cpg_name',
             'marca_detalle':'brand_name',
             'marca_generica':'brand_name',
             'gramaje':'package',
             'descripcion_nielsen':'product_name',
             'segmento':'category_en'}
for x,y in null_fill.items():
    aux.loc[:,x] = aux.loc[:,y]
    
df = df.append(aux,ignore_index=True)

In [19]:
for col in df.columns:
    df[col] = df[col].astype(str).str.replace('nan','')
    
df = df.drop(columns='cpg_name').fillna('')
df.sample()

,city_name,store_name,category_en,category_id,product_id,product_name,barcodes,package,product_sku,brand_filter,...,segmento,subsegmento_1,subsegmento_2,subsegmento_3,cpg_filter,mes,anio,brand_id,brand_name,cpg_id
3768,Ciudad de México,La Comer,Yogurt,24,1399194,Yoghurt bebible con fresa,7501020561173,6 x 220 g,,Lala (5096),...,BB LIQUIDO,BASE BUSINESS,,,Grupo Lala MX (7),12,2021,5096,Lala,7


In [20]:
brand_cols = ['formato','proveedor_general','proveedor_abierto','marca_detalle','marca_generica','gramaje','descripcion_nielsen','segmento','subsegmento_1','subsegmento_2','subsegmento_3']
display(df[brand_cols].sample())

for col in brand_cols:
    df[col] = df[col].str.strip().str.title()

display(df[brand_cols].sample())

,formato,proveedor_general,proveedor_abierto,marca_detalle,marca_generica,gramaje,descripcion_nielsen,segmento,subsegmento_1,subsegmento_2,subsegmento_3
3188,PACK,YAKULT,YAKULT,YAKULT 40,YAKULT,0.4,YAKULT 40LT 5PZAS 80GR C/U=400GR NAL,LACTEOS FERMENTADOS,LACTEOS FERMENTADOS,,


,formato,proveedor_general,proveedor_abierto,marca_detalle,marca_generica,gramaje,descripcion_nielsen,segmento,subsegmento_1,subsegmento_2,subsegmento_3
2464,Medio Galon,Califa Farms,,Califia Farms,,1.4,Califia Farms Pure Coconut Almondmilk Blend Bo...,Plant Based,Plant Based Refrigerada,Almendra,Fresca


In [21]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('mes')['sales'].sum().reset_index()
aux_fin['mes'] = aux_fin['mes'].astype(int).map(dict(zip(range(1,13),
                                               ['ene','feb','mar','abr','may','jun',
                                                'jul','ago','sep','oct','nov','dic'])))
aux_fin

,mes,sales
0,dic,13731504.00


### Marca genérica

In [22]:
df.loc[df[df['marca_generica']==''].index, 'marca_generica'] = df.loc[df[df['marca_generica']==''].index, 'marca_detalle']
df[df['marca_generica']=='']

,city_name,store_name,category_en,category_id,product_id,product_name,barcodes,package,product_sku,brand_filter,...,segmento,subsegmento_1,subsegmento_2,subsegmento_3,cpg_filter,mes,anio,brand_id,brand_name,cpg_id


## Export

### Same product_id as past

In [23]:
print(len(df))
if past:
    som = pd.ExcelFile(os.path.join('/'.join(BASE_DIR.split('/')[:-2]),
                                    f'SoM_{SOM_CPG}.xlsx')).parse(sheet_name=0)
    df = df.merge(som.drop_duplicates('product_id')[['product_id']].reset_index(drop=True).astype(str))
    
print(len(df))

16347
16347


In [24]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('mes')['sales'].sum().reset_index()
aux_fin['mes'] = aux_fin['mes'].astype(int).map(dict(zip(range(1,13),
                                               ['ene','feb','mar','abr','may','jun',
                                                'jul','ago','sep','oct','nov','dic'])))

In [25]:
aux = aux_ini.copy()
aux['month  '] = aux['month  '].str.split().str[0].str[:3]
aux = aux.set_index('month  ').join(aux_fin.set_index('mes'),rsuffix='_fin')
aux['dif'] = aux['sales_fin'] - aux['sales']
aux['dif_p'] = aux['sales_fin'] / (aux['sales']+1e-10) - 1

### Ordered columns

In [26]:
export = df[['city_name', 'store_name', 'category_en', 'category_id', 'product_id',
             'product_name', 'barcodes', 'package', 'product_sku', 'brand_name',
             'sales', 'orders', 'users', 'units_found', 'units_replaced',
             'units_requested', 'formato', 'proveedor_general', 'proveedor_abierto',
             'marca_detalle', 'marca_generica', 'gramaje', 'descripcion_nielsen',
             'segmento', 'subsegmento_1', 'subsegmento_2', 'subsegmento_3',
             'mes', 'anio']].copy()
export.sort_values(['anio','mes'],inplace=True)
export.sample(4)

,city_name,store_name,category_en,category_id,product_id,product_name,barcodes,package,product_sku,brand_name,...,marca_detalle,marca_generica,gramaje,descripcion_nielsen,segmento,subsegmento_1,subsegmento_2,subsegmento_3,mes,anio
8537,Monterrey,HEB,Desserts & Refrigerated Bakery,22,1010586,Gelatina sabor fresa,7506443100466,Doypach 125 g,,Dany,...,Dany Pouch,Dany,0.125,Danone Dany Gelatina De Agua Sabor Fresa Pouch...,Gelatinas,Gelatinas Regulares,,,12,2021
4262,Ciudad de México,Soriana,Water,35,1155782,Agua infusionada limón,758104006724,Botella 1 L,758104006724,Bonafont Levité,...,Levite,Levite,1.0,Bonafont Levite Limon S/Aa 1L,Bebidas Saborizadas,Agua Saborizada,,,12,2021
9339,Monterrey,Soriana,Yogurt,24,336478,Yoghurt griego con arándanos,7501032395605,Vaso 150 g,,Oikos,...,Oikos,Oikos,0.15,Danone Oikos Griego Solido Arandano Bot 150Gr Nal,Griego,Griego,,,12,2021
7794,Mérida,Chedraui,Yogurt,24,340328,Yoghurt doble cero sabor fresa,7501040090295,1 kg,,Yoplait,...,Yoplait Doble Cero,Yoplait Doble Cero,1.0,Yoplait Fresa Batido Light 0% Grasa 1000 Gr.Nal.,Light,Light,,,12,2021


In [27]:
export.to_csv(os.path.join(BASE_DIR,f'som_{CPG}.csv'),index=False,sep='\t',encoding='utf-16')

## End

In [28]:
aux

,sales,sales_fin,dif,dif_p
month,,,,
dic,13731504.00,13731504.00,0.00,0.00


In [29]:
time_exp(time.time() - start)
tono()

0 minutos con 2.61 segundos
